In [1]:
# going to implement the transformer model i.e., for the CHAT GPT 3 version of transformer
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 4 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32
n_head = 8
n_layer = 2
dropout = 0.2
# ------------

In [3]:
torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open(r'C:\Users\bhara\Downloads\input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [5]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [6]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        '''
        Here we are using a decoder only block where the key query and value come from random and we make the tril version
        but in original encoder to decoder network paper we apply the cross attention where in
        the key and value come from the encoder block to every decoder block and query is generated from the decoder block
        this is very useful when we don't want to have the generate own text or for a machine translation
        This is the only change from the original transformer
        This is exactly done in GPT decoder only
        for chat gpt we have 2 states pretraining (decoder only network and using subword chunks of data rather than our model)
        dmodel in paper = n_embd in our model nanoGPT focus on the pretraining stage
        and finetuning(but it generates documents but not question answering as it is decoder only network so we use finetuning
        it is actually to do align 2nd stage with 3 steps:
        )
        :param x:
        :return:
        '''
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei) #randomly add so that it shuts communication between them
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


In [7]:
class MultiHeadAttention(nn.Module):
    # The next thing is multi headed attention
    '''
    What is multiheaded attention? multiple self- attentions in parallel
    It is easy to be created in pytorch
    num_heads - no of heads we need to run in parallel
    head_size - head size of each head
    '''
    def __init__(self,num_heads,head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd,n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        #concatinate all the heads
        return torch.cat([h(x) for h in self.heads], dim=-1)

In [8]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
class FeedForward(nn.Module):
    '''
    a simple feed  forward linear layer followed by a non-linearity
    it is on a token level every token does it
    '''
    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd,4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd,n_embd),
            nn.Dropout(dropout)
        )
    def forward(self,x):
        return self.net(x)

In [11]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [12]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size,n_embd)
        '''
        now we have 4 heads run in paralled with each block size of 8 that is 32 vectors stacked up
        '''
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        # self.sa_heads = MultiHeadAttention(4, n_embd//4)
        # self.ffwd = FeedForward(n_embd)
        self.ln_f = nn.LayerNorm(n_embd) # last final layer in the block before linear and softmax
        self.lm_head = nn.Linear(n_embd,vocab_size)

    def forward(self, idx, targets=None):
        B,T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)  # (B,T,vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop_idx so that we get it to the last block_size tokens
            #so that we will never pass no more than the block_size elements
            idx_cond = idx[:,-block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [15]:
model = BigramLanguageModel(vocab_size)

In [16]:
m = model.to(device)

In [17]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [18]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 4.1686, val loss 4.1727
step 100: train loss 3.4813, val loss 3.5167
step 200: train loss 3.2523, val loss 3.2807
step 300: train loss 3.1093, val loss 3.1322
step 400: train loss 2.9601, val loss 2.9818
step 500: train loss 2.8592, val loss 2.8959
step 600: train loss 2.7755, val loss 2.8055
step 700: train loss 2.7455, val loss 2.7449
step 800: train loss 2.6906, val loss 2.7106
step 900: train loss 2.6529, val loss 2.6744
step 1000: train loss 2.6047, val loss 2.6273
step 1100: train loss 2.5985, val loss 2.6014
step 1200: train loss 2.5702, val loss 2.5936
step 1300: train loss 2.5534, val loss 2.5672
step 1400: train loss 2.5467, val loss 2.5477
step 1500: train loss 2.5194, val loss 2.5378
step 1600: train loss 2.5225, val loss 2.5148
step 1700: train loss 2.5198, val loss 2.5161
step 1800: train loss 2.4956, val loss 2.5043
step 1900: train loss 2.4977, val loss 2.5060
step 2000: train loss 2.4663, val loss 2.4891
step 2100: train loss 2.4725, val loss 2.4845


In [19]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


Iest ame,
Whe;
And ING CRAANCEUSTI:
Thet--Nof have andss;.

Thor ;
Full, athe youlle ningateseinds ostualld tit thaE heriss ide;d skfalld mallil hat lit.

I GIOfarThotepleTer tof y koalds sing yeamn.

Sgby arest!

Whier bang stans yot coude peseall wiiywsemape toad, nurothousreru ned fovend dot nost eind enotelw
Avcirstkfaih it kouch's,epfare. Chamy Bustilr.

RDUG:
E the thict,
On, bertche you mulseill geth.

GICINK:

RGEO:
Hreit winge'g inth novee;
Man bigt gon awrast,
Ind this hou:
Dary a'm mi
